<a href="https://colab.research.google.com/github/Diooonis2Syracuse/ML/blob/main/Dogs_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision as tv
import cv2
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import torch.nn.functional as F

In [ ]:
!unzip /content/training_set.zip
!unzip /content/test_set.zip

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, path_dir1:str, path_dir2:str):
    super().__init__()

    self.path_dir1 = path_dir1
    self.path_dir2 = path_dir2

    self.list_dir1_sort = sorted(os.listdir(path_dir1))
    self.list_dir2_sort = sorted(os.listdir(path_dir2))

  def __len__(self):
    return len(self.list_dir1_sort) + len(self.list_dir2_sort)

  def __getitem__(self, idx):

    if idx < len(self.list_dir1_sort):
      id_class = 0
      img_path = os.path.join(self.path_dir1,  self.list_dir1_sort[idx])
    else:
      id_class = 1
      idx -= len(self.list_dir1_sort)
      img_path = os.path.join(self.path_dir2,  self.list_dir2_sort[idx])

    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    img = img/255.0

    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img = img.transpose((2, 0, 1))

    t_img = torch.from_numpy(img)
    t_id_class = torch.tensor([id_class])

    return {'img': t_img, 'label': t_id_class}

In [ ]:
train_dogs = '/content/training_set/dogs'
train_cats = '/content/training_set/cats'
test_dogs = '/content/test_set/dogs'
test_cats = '/content/test_set/cats'

dataset_train = Dataset(train_dogs, train_cats)
dataset_test = Dataset(test_dogs, test_cats)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

8005
2023


In [ ]:
batch_size = 4

train_dl = DataLoader(dataset_train,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True
                      )
test_dl = DataLoader(dataset_test,
                     batch_size=batch_size,
                     shuffle=True,
                     drop_last=False
                     )

In [ ]:
class VGG16(nn.Module):
  def __init__(self, out_nc):
    super().__init__()

    self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
    self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

    self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

    self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

    self.activ = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(2, 2)

    self.flatten = nn.Flatten()

    self.fc1 = nn.Linear(7*7*512, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, out_nc)

  def forward(self, x):
    out = self.conv1_1(x)
    out = self.activ(out)
    out = self.conv1_2(out)
    out = self.activ(out)

    out = self.maxpool(out)

    out = self.conv2_1(out)
    out = self.activ(out)
    out = self.conv2_2(out)
    out = self.activ(out)

    out = self.maxpool(out)

    out = self.conv3_1(out)
    out = self.activ(out)
    out = self.conv3_2(out)
    out = self.activ(out)
    out = self.conv3_3(out)
    out = self.activ(out)

    out = self.maxpool(out)

    out = self.conv4_1(out)
    out = self.activ(out)
    out = self.conv4_2(out)
    out = self.activ(out)
    out = self.conv4_3(out)
    out = self.activ(out)

    out = self.maxpool(out)

    out = self.conv5_1(out)
    out = self.activ(out)
    out = self.conv5_2(out)
    out = self.activ(out)
    out = self.conv5_3(out)
    out = self.activ(out)

    out = self.maxpool(out)

    out = self.flatten(out)
    out = self.fc1(out)
    out = self.activ(out)
    out = self.fc2(out)
    out = self.activ(out)
    out = self.fc3(out)

    return out

In [ ]:
model = VGG16(1)

In [ ]:
model

VGG16(
  (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5_3): Conv2d(512, 51

In [ ]:
def accuracy(pred, label):
  answer = (F.sigmoid(pred.detach().cpu()).numpy() > 0.5) == (label.cpu().numpy() > 0.5)
  return answer.mean()

In [ ]:
for sample in train_dl:
  img = sample['img']
  label = sample['label']
  model(img)
  break

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(model))

134264641


In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [ ]:
num_epochs = 10
torch.manual_seed(1)

for epoch in range(num_epochs):
  accurace_train = 0

  for sample in (pbar := tqdm(train_dl)):
    x_batch, y_batch = sample['img'], sample['label'].float()
    pred = model(x_batch)
    loss = loss_fn(pred, y_batch)
    optimizer.zero_grad()
    loss.backward()
    loss_item = loss.item()
    pbar.set_description(f'Loss: {loss_item:.4f}')
    optimizer.step()
    is_correct = ((pred>=0.5).float() == y_batch).float()
    accurace_train += is_correct.sum()
  accurace_train /= len(train_dl.dataset)
  print(f'Эпоха {epoch + 1} - Точность {accurace_train * 100:.4f}%')

In [ ]:
torch.manual_seed(1)
accuracy_test = 0
with torch.no_grad():
  for sample in tqdm(test_dl):
    x_batch, y_batch = sample['img'], sample['label'].float()
    pred = model(x_batch)
    is_correct_test = (torch.argmax(pred, dim=1) == y_batch).float()
    accuracy_test += is_correct_test.sum()
accuracy_test /= len(test_dl.dataset)
print(f'Точность при тестировании: {accuracy_test * 100:.4f}%')

100%|██████████| 64/64 [00:12<00:00,  5.05it/s]

Точность при тестировании: 83.0944%
